In [3]:
import numpy as np
import pandas as pd
import math
from sympy import primerange

# --- Constants and structures ---
phi = (1 + math.sqrt(5)) / 2
pi = math.pi
sqrt2 = math.sqrt(2)
alpha = 1 / 137.035999
alpha_inv = 137.035999
e = math.e

# Full signature bank of constants and symbolic residues
target_constants = {
    "alpha": alpha,
    "alpha_inv": alpha_inv,
    "1-alpha": 1 - alpha,
    "1-alpha_inv": 1 - alpha_inv,
    "pi": pi,
    "1-pi": 1 - pi,
    "pi_inv": 1 / pi,
    "phi": phi,
    "1-phi": 1 - phi,
    "phi^2": phi**2,
    "sqrt2": sqrt2,
    "1-sqrt2": 1 - sqrt2,
    "sqrt2_inv": 1 / sqrt2,
    "e": e,
    "1-e": 1 - e,
    "e_inv": 1 / e,
}

# Add digit-string signatures to test for symbolic digit fragments
digit_signatures = [
    "7297", "36034", "297", "035", "137035", "1137", "000137", "734", "134", "729735", "1370", "730", "1415", "159"
]

# String to base-n conversion
def decimal_to_base(n, base, length=12):
    digits = []
    for _ in range(length):
        n *= base
        digit = int(n)
        digits.append(str(digit))
        n -= digit
    return "".join(digits)

# Entropy of digit string
def digit_entropy(s):
    if not s:
        return 0
    counts = [s.count(str(d)) for d in range(10)]
    freqs = [c / len(s) for c in counts if c > 0]
    return -sum(f * math.log(f, 2) for f in freqs)

# Fragment matcher: return how many substrings matched
def match_digit_signatures(s):
    return sum(1 for sig in digit_signatures if sig in s)

# Δ and folding: apply to constant difference
def fold_delta(val1, val2, base=9, powers=[1, 2, 3]):
    deltas = []
    delta = abs(val1 - val2)
    for p in powers:
        folded = delta / (base ** p)
        deltas.append((p, folded))
    return deltas

# Generate Fibonacci-style seed (e.g., 0.0100101001…)
def fibonacci_word(n):
    a, b = "0", "01"
    while len(b) < n:
        a, b = b, b + a
    return b[:n]

def fibword_to_decimal(s):
    return sum(int(b) / 2 ** (i + 1) for i, b in enumerate(s))

# Main test loop
records = []
fib_sequence = fibonacci_word(200)

for i in range(20, len(fib_sequence) - 12):
    bit_window = fib_sequence[i:i+12]
    dec_val = fibword_to_decimal(bit_window)

    row = {
        "bit_window": bit_window,
        "decimal_val": dec_val
    }

    # Δ between this and each target constant
    for name, const in target_constants.items():
        delta = abs(dec_val - const)
        row[f"delta_{name}"] = delta

        # Fold deltas over powers of base 9
        for p, folded in fold_delta(dec_val, const, base=9):
            row[f"delta_{name}_fold9^{p}"] = folded

    # base-n representations and entropy
    for base in [2, 3, 9, 10]:
        base_repr = decimal_to_base(dec_val, base)
        row[f"base{base}_repr"] = base_repr
        row[f"entropy_base{base}"] = digit_entropy(base_repr)
        row[f"sig_match_base{base}"] = match_digit_signatures(base_repr)

    records.append(row)

df_deep_zeta_probe = pd.DataFrame(records)

from IPython.display import display
display(df_deep_zeta_probe)

df_deep_zeta_probe.to_csv("results_alpha_probe_fixed.csv", index=False)



,bit_window,decimal_val,delta_alpha,delta_alpha_fold9^1,delta_alpha_fold9^2,delta_alpha_fold9^3,delta_alpha_inv,delta_alpha_inv_fold9^1,delta_alpha_inv_fold9^2,delta_alpha_inv_fold9^3,...,sig_match_base2,base3_repr,entropy_base3,sig_match_base3,base9_repr,entropy_base9,sig_match_base9,base10_repr,entropy_base10,sig_match_base10
0,001001010010,0.145020,0.137722,0.015302,0.001700,0.000189,136.890979,15.210109,1.690012,0.187779,...,0,010220201102,1.554585,0,126642282635,2.522055,0,145019531250,2.625815,0
1,010010100100,0.290039,0.282742,0.031416,0.003491,0.000388,136.745960,15.193996,1.688222,0.187580,...,0,021211102211,1.459148,0,254384575372,2.522055,0,290039062500,2.284159,0
2,100101001001,0.580322,0.573025,0.063669,0.007074,0.000786,136.455677,15.161742,1.684638,0.187182,...,0,120200001111,1.483356,0,520044035830,2.418296,1,580322265625,2.355389,0
3,001010010010,0.160645,0.153347,0.017039,0.001893,0.000210,136.875354,15.208373,1.689819,0.187758,...,0,011100002222,1.554585,0,140088072760,2.584963,0,160644531250,2.751629,0
4,010100100101,0.321533,0.314236,0.034915,0.003879,0.000431,136.714466,15.190496,1.687833,0.187537,...,0,022200101201,1.554585,0,280351830704,2.855389,1,321533203125,2.188722,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,010010010100,0.286133,0.278835,0.030982,0.003442,0.000382,136.749866,15.194430,1.688270,0.187586,...,0,021201120221,1.554585,0,251527633121,2.459148,0,286132812500,2.688722,0
164,100100101001,0.572510,0.565212,0.062801,0.006978,0.000775,136.463489,15.162610,1.684734,0.187193,...,0,120110100201,1.483356,0,513321141317,2.221252,0,572509765625,2.418296,0
165,001001010010,0.145020,0.137722,0.015302,0.001700,0.000189,136.890979,15.210109,1.690012,0.187779,...,0,010220201102,1.554585,0,126642282635,2.522055,0,145019531250,2.625815,0
166,010010100101,0.290283,0.282986,0.031443,0.003494,0.000388,136.745716,15.193968,1.688219,0.187580,...,0,021211121122,1.325011,0,254548350446,2.522055,0,290283203125,2.584963,0
